In [1]:
import pandas as pd
import json

In [2]:
def load_data():
    df = pd.read_parquet("sympy_2024_test_v01.parquet")

    def load_json(s):
        """load str(dict|list), please do json.dumps in future"""

        if s == '[None]':
            return []
        return json.loads(s.replace("'", '"'))

    for i in ['base_commit_ids', 'base_commit_dates', 'num_changed_files', 
              'changed_files_list', 'linked_issue_nums', 'linked_issue_date_open', 
              'linked_issue_date_closed']:
        #print(i)
        df[i] = df[i].apply(load_json)
    df['base_commit_dates'] = df['base_commit_dates'].apply(lambda x: [pd.to_datetime(i) for i in x])
    df['linked_issue_date_open']  = df['linked_issue_date_open'].apply(lambda x: [pd.to_datetime(i) for i in x])
    df['linked_issue_date_closed']  = df['linked_issue_date_closed'].apply(lambda x: [pd.to_datetime(i) for i in x])
    for column in ['pr_close_date', 'pr_open_date', 'resolving_commit_date']:
        #print(column)
        df[column] = pd.to_datetime(df[column])

    print(df.shape)
    print(df.keys())
    return df

#Load data
df = load_data()

n_with_issue = df['_linked_issue_desc'].notna().sum()
n = len(df)
print("issue linked %f [%d/%d]"%(n_with_issue/n, n_with_issue, n))
# print("")
# df = df[df['_linked_issue_desc'].notna()]

(533, 15)
Index(['repo_name', 'base_commit_ids', 'base_commit_dates',
       'resolving_commit_id', 'resolving_commit_date', 'pr_num',
       'pr_close_date', 'pr_open_date', 'num_changed_files',
       'changed_files_list', 'linked_issue_nums', '_linked_issue_desc',
       '_pr_description', 'linked_issue_date_open',
       'linked_issue_date_closed'],
      dtype='object')
issue linked 0.260788 [139/533]


/tmp/ipykernel_1827281/932331968.py:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[column] = pd.to_datetime(df[column])


In [3]:
group1 = ['base_commit_ids', 'base_commit_dates', 'num_changed_files', 'changed_files_list']
group2 = ['linked_issue_nums', 'linked_issue_date_open', 'linked_issue_date_closed']

df[group1].apply(lambda x: x.apply(len)).drop_duplicates(),\
df[group2].apply(lambda x: x.apply(len)).drop_duplicates()

(   base_commit_ids  base_commit_dates  num_changed_files  changed_files_list
 0                2                  2                  2                   2,
     linked_issue_nums  linked_issue_date_open  linked_issue_date_closed
 0                   0                       0                         0
 2                   1                       1                         1
 16                  1                       1                         0
 84                  2                       2                         2
 90                  3                       3                         3)

In [4]:
from copy import deepcopy

def explode_dataframe(df):
    """
        Explode dataframe by 2 groups
         if second groups len()==0 => it will have None
    """
    group1 = ['base_commit_ids', 'base_commit_dates', 'num_changed_files', 'changed_files_list']
    group2 = ['linked_issue_nums', 'linked_issue_date_open', 'linked_issue_date_closed']

    data_explode = []

    for orig_id, row in df.iterrows():
        dct_base = {k: v for k, v in  row.to_dict().items() if k not in group1+group2}
        n_group1 = len(row[group1[0]])
        
        #Case if second group is None)
        n_group2 = max(1, len(row[group2[0]]))
        for ind1 in range(n_group1):
            dct_base_commits = {k:row[k][ind1] for k in group1}
            for ind2 in range(n_group2):
                dct_linked_issue = {k: row[k][ind2] if len(row[k]) != 0 else None for k in group2}
                data_explode.append({**{"orig_id": orig_id}, **dct_base, **dct_base_commits, **dct_linked_issue})

    data_explode = pd.DataFrame(data_explode)
    return data_explode

data = explode_dataframe(df)
# Empty space
# del df

# Rename some columns according to SWE-bench dataset
# Add columns to have understandable names
# 'repo': Value(dtype='string', id=None)
# 'instance_id': Value(dtype='string', id=None)
# 'base_commit': Value(dtype='string', id=None)  # наш base_commit_before
# 'patch':      - правильный ответ 
# 'test_patch': Value(dtype='string', id=None), # какие тесты нужно подставить чтобы были саме новые тесты
# 'problem_statement': Value(dtype='string', id=None), # нужно посмотреть они еще добавляют данные с форума
# 'hints_text': Value(dtype='string', id=None),        # ? с форума ?
# 'created_at': Value(dtype='string', id=None),        # дата создания ?? чего
# 'version': Value(dtype='string', id=None),           # ? версия либы
# 'FAIL_TO_PASS': Value(dtype='string', id=None),      # тесты которые могут упасть (skipped, error)
# 'PASS_TO_PASS': Value(dtype='string', id=None),      # тесты которые должны быть passed
# 'environment_setup_commit': Value(dtype='string', id=None) # hash Dockerfile

data['base_commit_before'] = data['base_commit_ids']
data['base_commit_after'] = data['resolving_commit_id']
data['problem_statement'] = data['_linked_issue_desc']
data['created_at'] = data['linked_issue_date_open']

In [5]:
data['build_command'] = "pip install .\npip install -r requirements-dev.txt\npip install pytest-json-report"
data['test_command'] = "pytest --json-report --json-report-file=report_pytest.json"

In [6]:
# data['linked_issue_date_open']
data.shape

(1094, 22)

In [7]:
data = data[['orig_id', 
'repo_name', "base_commit_before", "base_commit_after",
"problem_statement", "created_at", 
"build_command",
"test_command",
'resolving_commit_id', 
'resolving_commit_date',
'pr_num', 
'pr_close_date', 
'pr_open_date', 
'_linked_issue_desc',
'_pr_description', 'base_commit_ids', 'base_commit_dates',
'num_changed_files', 'changed_files_list', 'linked_issue_nums',
'linked_issue_date_open', 'linked_issue_date_closed']]

In [8]:
print("data.shape", data.shape)
print("data['problem_statement'].notna().sum()", data['problem_statement'].notna().sum())
print("data['problem_statement'].dropna().nunique()", data['problem_statement'].dropna().nunique())

data.shape (1094, 22)
data['problem_statement'].notna().sum() 306
data['problem_statement'].dropna().nunique() 137


In [9]:
from draft_git_scraper_v02 import fetch_git_diff, get_test_patch, get_patch

In [10]:
from tqdm import tqdm
folder_with_sympy = "dynamic_bench/repos/sympy/"
data['full_patch'] = None
data['test_patch'] = None
data['patch'] = None

for ind, row in tqdm(data.iterrows()):
    data.loc[0, 'base_commit_before']
    data.loc[0, 'base_commit_after']

    git_diff = fetch_git_diff(folder=folder_with_sympy, 
                   base_commit_before=row['base_commit_before'], 
                   base_commit_after=row['base_commit_after']
                  )

    test_patch = get_test_patch(git_diff)
    patch = get_patch(git_diff)
    
    data.loc[ind, 'test_patch'] = test_patch
    data.loc[ind, 'patch'] = patch
    data.loc[ind, 'full_patch'] = git_diff

1094it [00:11, 98.04it/s] 


In [11]:
data.to_parquet("sympy_2024_test_v02_exploded_with_patch.parquet")

In [12]:
image_name = "rti-sympy--sympy--490c6d0f7df7b02a70e04c0ed4ead899cd430e70"

In [13]:
import sys
sys.path.insert(0, '/data/adam/git/repotest/')

from tqdm import tqdm

import repotest
from repotest import __version__
print(__version__)

from repotest.core.docker.python import PythonDockerRepo
from repotest.core.exceptions import GitException

DEFAULT_CACHE_FOLDER=/data/adam/.cache/repotest
0.3.29


In [14]:
task = data.loc[4].to_dict()

In [15]:
!ls dynamic_bench/repos/sympy/sympy/polys/

__init__.py	       galoistools.py		   polyquinticconst.py
agca		       groebnertools.py		   polyroots.py
appellseqs.py	       heuristicgcd.py		   polytools.py
benchmarks	       matrices			   polyutils.py
compatibility.py       modulargcd.py		   puiseux.py
constructor.py	       monomials.py		   rationaltools.py
densearith.py	       multivariate_resultants.py  ring_series.py
densebasic.py	       numberfields		   rings.py
densetools.py	       orderings.py		   rootisolation.py
dispersion.py	       orthopolys.py		   rootoftools.py
distributedmodules.py  partfrac.py		   solvers.py
domainmatrix.py        polyclasses.py		   specialpolys.py
domains		       polyconfig.py		   sqfreetools.py
euclidtools.py	       polyerrors.py		   subresultants_qq_zz.py
factortools.py	       polyfuncs.py		   tests
fglmtools.py	       polymatrix.py
fields.py	       polyoptions.py


In [16]:
# print(task['test_patch'])

In [17]:
repo = PythonDockerRepo(repo = task['repo_name'],
                        base_commit = task['base_commit_before'],
                        image_name = image_name
                       )

repo.clean()

In [18]:
# - clean
# - apply(test_patch)
# - apply(patch)
# - apply(test, test_patch)

In [19]:
data['test_files'] = data['changed_files_list'].apply(lambda x: [i for i in x if i.split('/')[-1].startswith("test_")])

In [20]:
data['small_test_command'] = data['test_files'].apply(lambda x: 'pytest ' + ' '.join(x) + " --json-report --json-report-file=report_pytest.json" if x else None)

In [21]:
data["_res_before"] = None
data["_res_test_patch"] = None
data["_res_patch"] = None
data["_res_full_patch"] = None

In [30]:
!du -sh {repo.cache_folder}

233M	/data/adam/.cache/repotest/6a36956e9fcf910159f93be330af79a228bc5421


In [45]:
import time
# time.sleep(1)

In [195]:
data['status'] = None

In [ ]:
for n_iteration, (ind, task) in tqdm(enumerate(data.iterrows()), 
                                     total = len(data)
                                    ):
    try:
        if task['small_test_command'] is None:
            continue
        elif task['test_patch'] == '':
            continue
        elif task['_res_full_patch'] is not None:
            continue
        elif len(task['changed_files_list']) > 100:
            continue

        print(ind)
        try:
            repo = PythonDockerRepo(repo = task['repo_name'],
                                    base_commit = task['base_commit_before'],
                                    image_name = image_name
                                   )
        except:
            print("Critical fail, git limit exceded")
            time.sleep(120)
            try:
                repo = PythonDockerRepo(repo = task['repo_name'],
                                        base_commit = task['base_commit_before'],
                                        image_name = image_name
                                       )
            except:
                time.sleep(600)
                data.loc[ind, "_res_before"] = "{}"
                continue

        # base_commit_before
        repo.clean()

        dct_before = repo.run_test(task['small_test_command'])
        data.loc[ind, "_res_before"] = json.dumps(dct_before)

        # base_commit_before + тесты из будущего
        repo.clean()
        repo.apply_patch(task['test_patch'])
        dct_test_patch = repo.run_test(task['small_test_command'], timeout=60*45)
        data.loc[ind, "_res_test_patch"] = json.dumps(dct_test_patch)

        # base_commit_before + фиксы файлов из будущего
        if task['patch'] != "":
            repo.clean()
            repo.apply_patch(task['patch'])
            dct_patch = repo.run_test(task['small_test_command'])
            data.loc[ind, "_res_patch"] = json.dumps(dct_patch)
        else:
            data.loc[ind, "_res_patch"] = json.dumps(dct_before)
        # base_commit_before + фиксы файлов из будущего + тесты из будущего (все равно что base_commit_after)
        repo.clean()
        if (task['patch'] != ''):
            repo.apply_patch(task['patch'])
        repo.apply_patch(task['test_patch'])
        dct_full_patch = repo.run_test(task['small_test_command'], timeout=60*45)
        data.loc[ind, "_res_full_patch"] = json.dumps(dct_full_patch)
        if (n_iteration % 50 == 0):
            data.to_parquet("data_250409.parquet")
        data.loc[ind, 'status'] = "ok"
    except:
        print("Critical fail")
        data.loc[ind, 'status'] = "fail"
        time.sleep(120)

  0%|                                                  | 0/1094 [00:00<?, ?it/s]

200


 18%|███████▎                                | 201/1094 [00:27<02:00,  7.43it/s]

215


 20%|███████▉                                | 216/1094 [00:49<03:53,  3.76it/s]

...

643


 59%|█████████████████████▏              | 644/1094 [2:19:27<9:10:43, 73.43s/it]

644


 59%|████████████████████▋              | 645/1094 [2:22:03<11:07:45, 89.23s/it]

647


In [ ]:
!pwd

In [205]:
# len(task['ch'])
# len(task['changed_files_list'])
# data['status'].value_counts()

In [206]:
# data[data['_res_before'].notna()]

In [207]:
# task['test_patch']
# !df -h .

In [ ]:
# task['patch']

In [ ]:
# data

In [ ]:
# repo = PythonDockerRepo(repo = task['repo_name'],
#                         base_commit = task['base_commit_before'],
#                         image_name = image_name
#                        )

In [ ]:
# repo.clean()

In [208]:
# Хорошие примеры:
#  - содержат problem_statement
#  - есть test:
#         before[test] = any
#         test_patch[test] = fail
#         patch[test] = any
#         full_path[test] = passed

In [185]:
data.to_parquet("data_250409.parquet")

In [209]:
# task['small_test_command']
# !df -h .

In [210]:
# data

def get_pass_fail_detail_stat(task):
    test_before = json.loads(task['_res_before'])
    test_test_patch = json.loads(task['_res_test_patch'])
    test_patch = json.loads(task['_res_patch'])
    test_full_patch = json.loads(task['_res_full_patch'])

    def get_test_dict(dct):
        res = {}
        for i in dct['pytest_json']['tests']:
            assert i['nodeid'] not in res
            res[i['nodeid']] = i['outcome']
        return res

    test_before = get_test_dict(test_before)
    test_test_patch = get_test_dict(test_test_patch)
    test_patch = get_test_dict(test_patch)
    test_full_patch = get_test_dict(test_full_patch)

    res = {}
    pass_to_pass = set()
    fail_to_pass = set()
    for test in (set(test_before.keys()) | set(test_test_patch.keys()) | set(test_patch.keys()) | set(test_full_patch.keys())):
        if not (test_before.get(test, None) == test_test_patch.get(test, None) == test_patch.get(test, None) == test_full_patch.get(test, None)):
            res[test] = {'\tbefore': test_before.get(test, None),
            'test_patch' : test_test_patch.get(test, None),
            'patch' : test_patch.get(test, None),
            'full_patch' : test_full_patch.get(test, None)}
            if (test_test_patch.get(test, None) != 'passed') and (test_full_patch.get(test, None) == 'passed'):
                pass_to_pass.add(test)
            if (test_test_patch.get(test, None) != 'passed') and (test_full_patch.get(test, None) != 'passed'):
                fail_to_pass.add(test)
    return {"pass_to_pass": pass_to_pass,
            "fail_to_pass": fail_to_pass,
            "details": res
           }

In [211]:
df35 = data[data['_res_full_patch'].notna()]

In [214]:
df35['base_commit_before'].nunique()

305

In [213]:
df35['PASS_TO_PASS'] = ""
df35['FAIL_TO_PASS'] = ""
df35['details'] = ""

/tmp/ipykernel_1827281/223995053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df35['PASS_TO_PASS'] = ""
/tmp/ipykernel_1827281/223995053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df35['FAIL_TO_PASS'] = ""
/tmp/ipykernel_1827281/223995053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [215]:
for ind, task in df35.iterrows():
    try:
        tmp = get_pass_fail_detail_stat(task)
    except:
        continue
    
    df35.loc[ind, 'PASS_TO_PASS'] = json.dumps(list(tmp.get('pass_to_pass', [])))
    df35.loc[ind, 'FAIL_TO_PASS'] = json.dumps(list(tmp.get('fail_to_pass', [])))
    df35.loc[ind, 'details']      = json.dumps(tmp.get('details'))

In [216]:
df35.to_parquet(f"{len(df35)}_examples.parquet")

In [160]:
# df35[column][:3].apply(json.loads)
df35['PASS_TO_PASS'][:4]#.apply(json.loads)

0          ["sympy/core/tests/test_arit.py::test_Mod"]
1    ["sympy/polys/numberfields/tests/test_subfield...
4    ["sympy/polys/numberfields/tests/test_subfield...
7                                                     
Name: PASS_TO_PASS, dtype: object

In [161]:
for column in ['_res_before', '_res_test_patch', '_res_patch', 
               '_res_full_patch', 'PASS_TO_PASS', 'FAIL_TO_PASS', 'details']:
    print(column)
    df35[column] = df35[column].apply(lambda x: json.loads(x) if x else None)
               

_res_before
_res_test_patch
_res_patch
_res_full_patch


/tmp/ipykernel_1827281/2995188601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df35[column] = df35[column].apply(lambda x: json.loads(x) if x else None)


PASS_TO_PASS
FAIL_TO_PASS
details


In [ ]:
# df35

In [166]:
# print("N=", df35.shape[0])
# print("base_commit.nunique", df)
df35[df35['problem_statement'].notna()]

,orig_id,repo_name,base_commit_before,base_commit_after,problem_statement,created_at,build_command,test_command,resolving_commit_id,resolving_commit_date,...,patch,test_files,small_test_command,_res_before,_res_test_patch,_res_patch,_res_full_patch,PASS_TO_PASS,FAIL_TO_PASS,details
4,2,sympy/sympy,490c6d0f7df7b02a70e04c0ed4ead899cd430e70,9173d613a048493c1a4bfb07e69a419b1c73aa3d,Issue #27798: Here is a reproduction.\n```pyth...,2025-03-20 15:28:03+00:00,pip install .\npip install -r requirements-dev...,pytest --json-report --json-report-file=report...,9173d613a048493c1a4bfb07e69a419b1c73aa3d,2025-03-22 11:46:24+00:00,...,diff --git a/sympy/polys/numberfields/subfield...,[sympy/polys/numberfields/tests/test_subfield.py],pytest sympy/polys/numberfields/tests/test_sub...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,[sympy/polys/numberfields/tests/test_subfield....,[],{'sympy/polys/numberfields/tests/test_subfield...
11,5,sympy/sympy,84f0a73189d94afb813c75f49c3c0470b567f848,23c84d5bf9541778873e4f3f35841b70e45d4049,Issue #27777: In the documentation we can enco...,2025-03-16 19:58:55+00:00,pip install .\npip install -r requirements-dev...,pytest --json-report --json-report-file=report...,23c84d5bf9541778873e4f3f35841b70e45d4049,2025-03-17 21:23:29+09:00,...,diff --git a/.github/workflows/runtests.yml b/...,"[bin/test_pytorch.py, sympy/printing/tests/tes...",pytest bin/test_pytorch.py sympy/printing/test...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,[sympy/printing/tests/test_torch.py::test_torc...,[sympy/printing/tests/test_torch.py::test_torc...,{'sympy/printing/tests/test_torch.py::test_tor...
32,16,sympy/sympy,4d8872f0cb9b5a32db80a80d98c8c4b9d11e3b37,bfdb8d51c1064a51a9ba050c608b92ba7de9eabc,Issue #22014: from this SO [post](https://stac...,2021-09-03 11:26:02+00:00,pip install .\npip install -r requirements-dev...,pytest --json-report --json-report-file=report...,bfdb8d51c1064a51a9ba050c608b92ba7de9eabc,2025-03-09 13:14:25-07:00,...,diff --git a/sympy/assumptions/handlers/sets.p...,[sympy/assumptions/tests/test_query.py],pytest sympy/assumptions/tests/test_query.py -...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,[sympy/assumptions/tests/test_query.py::test_r...,[],{'sympy/assumptions/tests/test_query.py::test_...
33,16,sympy/sympy,29b12a274115383dcb3be85ee95d14dee9e365df,bfdb8d51c1064a51a9ba050c608b92ba7de9eabc,Issue #22014: from this SO [post](https://stac...,2021-09-03 11:26:02+00:00,pip install .\npip install -r requirements-dev...,pytest --json-report --json-report-file=report...,bfdb8d51c1064a51a9ba050c608b92ba7de9eabc,2025-03-09 13:14:25-07:00,...,diff --git a/bin/test b/bin/test\nindex 783827...,"[sympy/assumptions/tests/test_query.py, sympy/...",pytest sympy/assumptions/tests/test_query.py s...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,{'stdout': '============================= test...,[sympy/polys/matrices/tests/test_fflu.py::test...,[sympy/polys/matrices/tests/test_xxm.py::test_...,{'sympy/physics/mechanics/tests/test_actuator....
36,18,sympy/sympy,ddea9b09a62294b99a63d8ceafa13508efb61c32,b836671fe8459e9301c620117b660c6c8ca20264,Issue #7286: Copy paste from the mailing list:...,2014-03-17 00:22:51+00:00,pip install .\npip install -r requirements-dev...,pytest --json-report --json-report-file=report...,b836671fe8459e9301c620117b660c6c8ca20264,2025-03-07 17:32:26-08:00,...,diff --git a/sympy/assumptions/handlers/sets.p...,[sympy/assumptions/tests/test_query

In [167]:
df35[df35['problem_statement'].notna()].to_json('13_examples.jsonl', 
                                                lines = True,
                                                orient = 'records'
                                               )

In [169]:
!jsonl2html --index_column=problem_statement 13_examples.jsonl

INFO:root:OK. Save results to 13_examples.html


In [219]:
# get_test_dict(test_before)

In [220]:
# b = json.loads(data.loc[0, '_res_test_patch'])['pytest_json']['tests']
# [(i['nodeid'], i['outcome']) for i in a['tests']]